**Step 1: Environment Setup & Data Preparation**

This block handles the initial setup. It imports necessary libraries, creates directories for organizing outputs (`charts/` and `dashboard_data/`), loads the dataset, standardizes the date format, and fills missing values using linear interpolation to ensure a continuous time series.


In [34]:
import pandas as pd
import numpy as np
import os
import warnings

# --- 1. Setup Directories & Settings ---
# Create folders to store charts and dashboard logs
os.makedirs('charts', exist_ok=True)
os.makedirs('dashboard_data', exist_ok=True)
warnings.filterwarnings("ignore")

# --- 2. Data Loading ---
file_path = "Daily_Public_Transport_Passenger_Journeys_by_Service_Type_20250603.csv"
df = pd.read_csv(file_path)

# Standardize 'Date' and set as index
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y', errors='coerce')
df = df.set_index('Date').sort_index()

# --- 3. Imputation Strategy ---
# Use Linear Interpolation to fill small gaps while preserving trends
df['Other'] = df['Other'].interpolate(method='linear').fillna(0)

# Calculate 'Total Journeys' (Sum of all service types)
df['Total Journeys'] = df[['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School', 'Other']].sum(axis=1)

print("Step 1 Complete: Data loaded, cleaned, and imputed.")

Step 1 Complete: Data loaded, cleaned, and imputed.


**Step 2: Exploratory Data Analysis (EDA)**

This block calculates the 5 key insights (Service Type split, Weekly Seasonality, School Impact, etc.), generates visualizations for each using matplotlib, and saves the underlying data to CSV files for your future dashboard.


In [35]:
import matplotlib.pyplot as plt

# --- Insight 1: Ridership by Service Type ---
total_journeys = (df[['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School', 'Other']].sum() / 1000000).sort_values(ascending=True)

plt.figure(figsize=(10, 6))
total_journeys.plot(kind='barh', color='skyblue')
plt.title('Total Passenger Journeys by Service Type (Millions)')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('charts/insight1_service_types.png')
plt.close()
# Save log
total_journeys.to_csv('dashboard_data/log_insight1_services.csv')

# --- Insight 2: Weekly Seasonality ---
df['DayOfWeek'] = df.index.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
avg_journeys = df.groupby('DayOfWeek')['Total Journeys'].mean().reindex(day_order)

plt.figure(figsize=(9, 5))
avg_journeys.plot(kind='bar', color='darkorange')
plt.title('Average Daily Journeys by Day of Week')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('charts/insight2_dayofweek.png')
plt.close()
# Save log
avg_journeys.to_csv('dashboard_data/log_insight2_dayofweek.csv')

# --- Insight 3: School Ridership Share ---
school_total = df['School'].sum()
total_all = df['Total Journeys'].sum()
school_impact = pd.Series({'School': school_total, 'Public': total_all - school_total})

plt.figure(figsize=(8, 8))
plt.pie(school_impact, labels=school_impact.index, autopct='%1.1f%%', startangle=90, colors=['gold', 'lightgray'])
plt.title('School vs. Public Ridership Share')
plt.savefig('charts/insight3_school_impact.png')
plt.close()
# Save log
school_impact.to_csv('dashboard_data/log_insight3_school.csv')

# --- Insight 4: Yearly Trend ---
yearly_trend = df['Total Journeys'].resample('Y').sum() / 1000000

plt.figure(figsize=(10, 5))
plt.plot(yearly_trend.index.year, yearly_trend.values, marker='o', color='green')
plt.title('Annual Ridership Trend (Millions)')
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig('charts/insight4_yearly_trend.png')
plt.close()
# Save log
yearly_trend.to_csv('dashboard_data/log_insight4_trend.csv')

print("Step 2 Complete: EDA charts saved to 'charts/' and logs to 'dashboard_data/'.")

Step 2 Complete: EDA charts saved to 'charts/' and logs to 'dashboard_data/'.


**Step 3: Multi-Model Forecasting (Prophet, SARIMA, LightGBM)**

This block generates the official 7-day forecast using three different algorithms:

- **Prophet:** Good for seasonality and holidays.  
- **SARIMA:** Statistical method excellent for short-term patterns.  
- **LightGBM:** A powerful machine learning model (added as requested).

It saves the forecast results for all three models to separate CSV files.


In [36]:
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import numpy as np

# --- Helper: Metrics Calculation ---
def calculate_metrics(y_true, y_pred):
    """Calculates RMSE, MAE, MAPE, and SMAPE."""
    y_true, y_pred = np.array(y_true, dtype=float), np.array(y_pred, dtype=float)
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    
    # SMAPE: Symmetric Mean Absolute Percentage Error
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape = np.mean(np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)) * 100
    
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'SMAPE': smape}

# --- Helper: LightGBM Feature Engineering ---
def create_features(data, label=None):
    X = data.copy()
    X['dayofweek'] = X.index.dayofweek
    X['month'] = X.index.month
    X['year'] = X.index.year
    X['dayofyear'] = X.index.dayofyear
    X['lag_7'] = X[label].shift(7)
    X['lag_14'] = X[label].shift(14)
    X['roll_mean_7'] = X[label].shift(7).rolling(window=7).mean()
    return X.dropna()

service_types = ['Local Route', 'Light Rail', 'Peak Service', 'Rapid Route', 'School']
start_date = df.index.max() + pd.Timedelta(days=1)
end_date = df.index.max() + pd.Timedelta(days=7)
future_range = pd.date_range(start_date, end_date, freq='D')

# Initialize Forecast DataFrames
forecast_p = pd.DataFrame({'Date': future_range}).set_index('Date')
forecast_s = pd.DataFrame({'Date': future_range}).set_index('Date')
forecast_l = pd.DataFrame({'Date': future_range}).set_index('Date')

print("--- Step 3: Generating 7-Day Forecasts (3 Models) ---")

for service in service_types:
    print(f"Processing {service}...")
    ts = df[service].astype(float)
    
    # --- MODEL A: PROPHET ---
    df_p = pd.DataFrame({'ds': ts.index, 'y': ts.values})
    m_p = Prophet(growth='linear', seasonality_mode='multiplicative', weekly_seasonality=True)
    m_p.fit(df_p)
    # Forecast
    future_p = m_p.make_future_dataframe(periods=7, include_history=False)
    pred_p = m_p.predict(future_p)['yhat'].clip(lower=0).round(0).astype(int)
    forecast_p[service] = pred_p.values

    # --- MODEL B: SARIMA ---
    try:
        m_s = SARIMAX(ts, order=(1, 1, 1), seasonal_order=(0, 1, 1, 7), 
                      enforce_stationarity=False, enforce_invertibility=False)
        res_s = m_s.fit(disp=False)
        pred_s = res_s.predict(start=start_date, end=end_date).clip(lower=0).round(0).astype(int)
        forecast_s[service] = pred_s.values
    except:
        forecast_s[service] = 0
    
    # --- MODEL C: LIGHTGBM ---
    df_lgb = create_features(pd.DataFrame(ts), label=service)
    X_train = df_lgb.drop(columns=[service])
    y_train = df_lgb[service]
    
    model_lgb = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, verbose=-1)
    model_lgb.fit(X_train, y_train)
    
    # Future features construction
    future_feats = []
    for date in future_range:
        hist_date = date - pd.Timedelta(days=7)
        lag_val = ts.loc[hist_date] if hist_date in ts.index else 0
        lag_14_val = ts.loc[date - pd.Timedelta(days=14)] if (date - pd.Timedelta(days=14)) in ts.index else 0
        roll_mean = ts.loc[date - pd.Timedelta(days=13):hist_date].mean()
        future_feats.append({
            'dayofweek': date.dayofweek, 'month': date.month, 
            'year': date.year, 'dayofyear': date.dayofyear,
            'lag_7': lag_val, 'lag_14': lag_14_val, 'roll_mean_7': roll_mean
        })
    
    X_future = pd.DataFrame(future_feats)[X_train.columns]
    pred_l = model_lgb.predict(X_future)
    forecast_l[service] = pred_l.clip(min=0).round(0).astype(int)

# Save Logs
forecast_p.to_csv('dashboard_data/log_forecast_prophet.csv')
forecast_s.to_csv('dashboard_data/log_forecast_sarima.csv')
forecast_l.to_csv('dashboard_data/log_forecast_lightgbm.csv')

print("Forecasts generated successfully.")

--- Step 3: Generating 7-Day Forecasts (3 Models) ---
Processing Local Route...
Processing Light Rail...
Processing Peak Service...
Processing Rapid Route...
Processing School...
Forecasts generated successfully.


**Step 4: Final Model Comparison (Detailed Backtesting)**

This block performs the rigorous "Face-Off" on the last 7 days of known data, evaluating models with multiple error metrics:

- **MAE:** Mean Absolute Error  
- **RMSE:** Root Mean Squared Error  
- **MAPE:** Mean Absolute Percentage Error  
- **SMAPE:** Symmetric MAPE  

- **Consolidated Results:** Saved as `log_model_comparison_full.csv`.  



In [37]:
print("\n--- Step 4: Final Model Comparison (Backtesting with Advanced Metrics) ---")
print("Evaluating forecast accuracy on the last 7 days of known data...\n")

# Define Backtest Split
test_horizon = 7
train_df = df.iloc[:-test_horizon]
test_df = df.iloc[-test_horizon:]

results = []
score = {'Prophet': 0, 'SARIMA': 0, 'LightGBM': 0}

# Print Header
print(f"{'Service':<15} | {'Model':<10} | {'MAE':<8} | {'RMSE':<8} | {'MAPE':<8} | {'SMAPE':<8}")
print("-" * 75)

for service in service_types:
    y_train = train_df[service].astype(float)
    y_test = test_df[service].astype(float)
    
    # ---------------- PROPHET ----------------
    p_df = pd.DataFrame({'ds': y_train.index, 'y': y_train.values})
    m_p = Prophet(growth='linear', seasonality_mode='multiplicative', weekly_seasonality=True)
    m_p.fit(p_df)
    future_p = m_p.make_future_dataframe(periods=test_horizon, include_history=False)
    y_pred_p = m_p.predict(future_p)['yhat'].values
    met_p = calculate_metrics(y_test, y_pred_p)
    
    # ---------------- SARIMA ----------------
    try:
        m_s = SARIMAX(y_train, order=(1, 1, 1), seasonal_order=(0, 1, 1, 7), 
                      enforce_stationarity=False, enforce_invertibility=False)
        fit_s = m_s.fit(disp=False)
        y_pred_s = fit_s.predict(start=len(y_train), end=len(y_train)+test_horizon-1).values
        met_s = calculate_metrics(y_test, y_pred_s)
    except:
        met_s = {'RMSE': 999, 'MAE': 999, 'MAPE': 999, 'SMAPE': 999} # Penalty

    # ---------------- LIGHTGBM ----------------
    df_lgb = create_features(pd.DataFrame({'y': y_train}), label='y')
    X_t = df_lgb.drop(columns=['y'])
    y_t = df_lgb['y']
    model_lgb = lgb.LGBMRegressor(n_estimators=100, verbose=-1)
    model_lgb.fit(X_t, y_t)
    
    future_feats_lgb = []
    for date in y_test.index:
        hist_date = date - pd.Timedelta(days=7)
        lag_val = y_train.loc[hist_date] if hist_date in y_train.index else 0
        lag_14_val = y_train.loc[date - pd.Timedelta(days=14)] if (date - pd.Timedelta(days=14)) in y_train.index else 0
        roll_mean = y_train.loc[date - pd.Timedelta(days=13):hist_date].mean()
        future_feats_lgb.append({
            'dayofweek': date.dayofweek, 'month': date.month, 'year': date.year, 'dayofyear': date.dayofyear,
            'lag_7': lag_val, 'lag_14': lag_14_val, 'roll_mean_7': roll_mean
        })
    X_test_lgb = pd.DataFrame(future_feats_lgb)[X_t.columns]
    y_pred_l = model_lgb.predict(X_test_lgb)
    met_l = calculate_metrics(y_test, y_pred_l)

    # --- Compare & Score ---
    # We use MAE as the tie-breaker for the "Winner"
    maes = {'Prophet': met_p['MAE'], 'SARIMA': met_s['MAE'], 'LightGBM': met_l['MAE']}
    winner = min(maes, key=maes.get)
    score[winner] += 1
    
    # Store full results
    results.append({'Service': service, 'Model': 'Prophet', **met_p, 'Is_Winner': (winner=='Prophet')})
    results.append({'Service': service, 'Model': 'SARIMA', **met_s, 'Is_Winner': (winner=='SARIMA')})
    results.append({'Service': service, 'Model': 'LightGBM', **met_l, 'Is_Winner': (winner=='LightGBM')})

    # Print Row per Model
    print(f"{service:<15} | {'Prophet':<10} | {met_p['MAE']:<8.0f} | {met_p['RMSE']:<8.0f} | {met_p['MAPE']:<8.2f} | {met_p['SMAPE']:<8.2f}")
    print(f"{'':<15} | {'SARIMA':<10} | {met_s['MAE']:<8.0f} | {met_s['RMSE']:<8.0f} | {met_s['MAPE']:<8.2f} | {met_s['SMAPE']:<8.2f}")
    print(f"{'':<15} | {'LightGBM':<10} | {met_l['MAE']:<8.0f} | {met_l['RMSE']:<8.0f} | {met_l['MAPE']:<8.2f} | {met_l['SMAPE']:<8.2f}")
    print("-" * 75)

print(f"\nFinal Score -> Prophet: {score['Prophet']}, SARIMA: {score['SARIMA']}, LightGBM: {score['LightGBM']}")
champion = max(score, key=score.get)
print(f">>> OVERALL CHAMPION: {champion} <<<")

# Save Detailed Log
pd.DataFrame(results).to_csv('dashboard_data/log_model_comparison_full.csv', index=False)


--- Step 4: Final Model Comparison (Backtesting with Advanced Metrics) ---
Evaluating forecast accuracy on the last 7 days of known data...

Service         | Model      | MAE      | RMSE     | MAPE     | SMAPE   
---------------------------------------------------------------------------
Local Route     | Prophet    | 10236    | 11838    | 998.98   | 199.11  
                | SARIMA     | 9182     | 10684    | 848.99   | 199.35  
                | LightGBM   | 9171     | 10674    | 815.23   | 199.27  
---------------------------------------------------------------------------
Light Rail      | Prophet    | 8272     | 8628     | 26036373726048940032.00 | 189.53  
                | SARIMA     | 5307     | 5935     | 15834479632497162240.00 | 186.58  
                | LightGBM   | 7156     | 7959     | 20855277400785137664.00 | 189.53  
---------------------------------------------------------------------------
Peak Service    | Prophet    | 219      | 252      | 772050394695950080.00